In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib as plot
import matplotlib.pyplot as plt
import os
from shapely.geometry import Point
import numpy as np
from osgeo import gdal, os, ogr, osr
from shapely.geometry import Polygon
import matplotlib.colors as colors
import pysheds
from pysheds.grid import Grid
import fiona
from fiona.crs import from_epsg
from fiona.crs import from_string



c:\Users\RIPHAWK\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.8.0' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
c:\Users\RIPHAWK\anaconda3\lib\site-packages\pandas\core\arrays\masked.py:62: UserWarning: Pandas requires version '1.3.4' or newer of 'bottleneck' (version '1.3.2' currently installed).
  from pandas.core import (


### In this Notebook we:
- Delineate a catchment for each heatsource model node
- Exctract the area of valley, hillslope, and ridgetop in each catchment
- Determine the baseflow of each node based on Juckems baseflow model
- Determining the amount of accretion coming into the river between nodes, for use in heatsource tempreature model.

In [33]:
#reading in data with relative paths so that it can be used by others who clone repository from github
script_dir = os.getcwd()
script_dir
# need to use '..' to go up to parent directory to access observations
# Path to model nodes csv with lat lon of each node
nodes_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'satellite_update','nodes', 'WFK_100m_nodes.csv'))

#Path to filled & burned DEM
dem_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'tif','WFK_demsubset4326_fillburn.tif'))

#Folder to put shapefiles for catchments of each node
catchments_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'accretion_model', 'shp', 'node_catchments'))

#Path to SSURGO polygons for landform extraction
landforms_path = os.path.abspath(os.path.join(script_dir, '..', '..', 'accretion_model', 'shp', 'WFK_SSURGO', 'WFK_landformsFINAL6608.shp'))
landforms_out_csv = os.path.abspath(os.path.join(script_dir, '..', '..', 'accretion_model', 'Nodes_landforms_result.csv'))

#Final Accretion csv
accretion_csv = os.path.abspath(os.path.join(script_dir, '..', '..', 'accretion_model', 'accretion_results.csv'))


### Delineating catchments for each Heatsource Node


In [17]:
def batch_catchment_delineation(burneddem, pointslist):
    # Read the points list CSV file into a DataFrame
    pointslist = pd.read_csv(pointslist)
    # Iterate through each point in the points list
    for index, row in pointslist.iterrows():
        # Create a grid the size of the DEM
        grid = Grid.from_raster(burneddem, data_name='dem')
        # Read the DEM raster data
        dem = grid.read_raster(burneddem)

        # Hydrocondition the DEM
        # Fill pits in the DEM
        pit_filled_dem = grid.fill_pits(dem)
        # Fill depressions in the DEM
        flooded_dem = grid.fill_depressions(pit_filled_dem)
        # Resolve flat areas in the DEM
        inflated_dem = grid.resolve_flats(flooded_dem)
        # Compute flow direction with a specified direction map
        dirmap = (64, 128, 1, 2, 4, 8, 16, 32)
        fdir = grid.flowdir(inflated_dem, dirmap=dirmap)
        # Calculate flow accumulation
        acc = grid.accumulation(fdir, dirmap=dirmap)
        # Delineate a catchment for each point

        # Get the coordinates and name from the points list
        x, y = row['LONGITUDE'], row['LATITUDE']
        name = str(row['NODE_ID'])
        # Snap the pour point to the nearest high accumulation point
        x_snap, y_snap = grid.snap_to_mask(acc > 1000, (x, y))

        # Delineate the catchment using the snapped coordinates
        catch = grid.catchment(x=x_snap, y=y_snap, fdir=fdir, dirmap=dirmap, 
                               xytype='coordinate')

        # Crop and export the catchment as a shapefile
        grid.clip_to(catch)
        catch_view = grid.view(catch, dtype=np.uint8)
        shapes = grid.polygonize(catch_view)

        # Specify the schema for the shapefile
        schema = {'geometry': 'Polygon',
                  'properties': {'LABEL': 'float:16'}}
        # Define the Proj4 string for EPSG 4326 (WGS 84)
        crs_proj4 = '+proj=longlat +ellps=WGS84 +datum=WGS84 +no_defs'
        # Create a CRS object from the Proj4 string
        crs = from_string(crs_proj4)

        # Write the shapefile with the catchment data
        with fiona.open(os.path.join(catchments_path, 'autocatchment4326_' + str(name) + '.shp'), 'w',
                        driver='ESRI Shapefile',
                        crs=crs,
                        schema=schema) as c:
            i = 0
            # Iterate through each shape and value to write them to the shapefile
            for shape, value in shapes:
                rec = {}
                rec['geometry'] = shape
                rec['properties'] = {'LABEL': str(value)}
                rec['id'] = str(i)
                c.write(rec)
                i += 1


In [18]:
batch_catchment_delineation(burneddem = dem_path,
                                     pointslist = nodes_path)



### Finding the amount of landform classes in each catchment

In [26]:
import os
import geopandas as gpd
import pandas as pd
from pyproj import CRS
from shapely.geometry import box

# Specify the folder containing the larger shapefiles
larger_shapefile_path = landforms_path

# Specify the folder containing the smaller shapefile for clipping
smaller_shapefiles_folder = catchments_path

#Specify the target CRS (EPSG 6608)
target_crs = CRS.from_epsg(6608)

# Read the larger shapefile
larger_gdf = gpd.read_file(larger_shapefile_path)

# Reproject the larger shapefile to the target CRS
larger_gdf = larger_gdf.to_crs(target_crs)

# Create an empty DataFrame to store the results
results_df = pd.DataFrame(columns=['SmallerShapefile'])

# Loop through smaller shapefiles in the folder
for filename in os.listdir(smaller_shapefiles_folder):
    if filename.endswith(".shp"):
        # Construct the full path to the smaller shapefile
        smaller_shapefile_path = os.path.join(smaller_shapefiles_folder, filename)
        print(smaller_shapefile_path)
        # Read the smaller shapefile
        smaller_gdf = gpd.read_file(smaller_shapefile_path)

        # Reproject the smaller shapefile to the target CRS
        smaller_gdf = smaller_gdf.to_crs(target_crs)

        # Clip the larger shapefile with the smaller shapefile
        clipped_gdf = gpd.overlay(larger_gdf, smaller_gdf, how='intersection')

        # Get a list of unique classes from the clipped result
        unique_classes = clipped_gdf['landform_2'].unique()

        # Create a dictionary to store class areas
        class_areas = {'SmallerShapefile': filename}

        # Calculate areas for each class within the clipped areas
        for class_name in unique_classes:
            class_gdf = clipped_gdf[clipped_gdf['landform_2'] == class_name]
            class_area = class_gdf.geometry.area.sum() / 1000000  # Convert to square kilometers
            class_areas[class_name] = class_area

        # Add results to the DataFrame
        results_df = pd.concat([results_df, pd.DataFrame([class_areas])], ignore_index=True)

# Fill any NaN values in the DataFrame with 0
results_df = results_df.fillna(0)

# Select every third row starting from the second
#selected_rows = results_df.iloc[1::3]

# Calculate the total area for each row
#selected_rows['TotalArea'] = selected_rows.iloc[:, 1:].sum(axis=1)
results_df['TotalArea'] = results_df.iloc[:, 1:].sum(axis=1)

# Rename the columns if necessary
# selected_rows.rename(columns={'1': 'Valley', '2': 'Hillside', '3': 'Ridgetop'}, inplace=True)

# Save the results DataFrame to a CSV file
results_df.to_csv(landforms_out_csv, index=False)

print('Class areas within the clipped areas have been calculated and saved to landforms_out_csv')

d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp\node_catchments\autocatchment4326_0.0.shp
d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp\node_catchments\autocatchment4326_1.0.shp
d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp\node_catchments\autocatchment4326_10.0.shp
d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp\node_catchments\autocatchment4326_100.0.shp
d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp\node_catchments\autocatchment4326_101.0.shp
d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp\node_catchments\autocatchment4326_102.0.shp
d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp\node_catchments\autocatchment4326_103.0.shp
d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp\node_catchments\autocatchment4326_104.0.shp
d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp\node_catchments\autocatchment4326_105.0.shp
d:\Ben_wd\THESIS\heatsource\post_defense\accretion_model\shp

### Joining Discharge data and Landform proportions for baseflow estimate calculations

In [44]:
df = pd.read_csv(landforms_out_csv)
df.rename(columns={'SmallerShapefile': 'Site', '1' : 'Valley', '2': 'Hillside', '3': 'Ridgetop'}, inplace=True)
df['number'] = df['Site'].str.extract(r'_(\d+(\.\d+)?)\.shp$')[0].astype(float)
df["baseflow_juckem"] = (df["Valley"]*20.1*.0003169)+(df["Hillside"]*36.3*.0003)+(df["Ridgetop"]*16*.0003)
df_sorted_by_area = df.sort_values(by='number', ascending = False)
df_sorted_by_area['accretion'] = df_sorted_by_area['baseflow_juckem'].diff()
df_sorted_by_area.to_csv(accretion_csv)

df_sorted_by_area


,Site,Valley,Hillside,Ridgetop,TotalArea,number,baseflow_juckem,accretion
96,autocatchment4326_185.0.shp,3.663581,6.381013,42.478311,52.522905,185.0,0.296721,NaN
95,autocatchment4326_184.0.shp,3.695816,6.425455,42.508879,52.630150,184.0,0.297557,0.000836
94,autocatchment4326_183.0.shp,3.707663,6.434004,42.512530,52.654196,183.0,0.297743,0.000186
93,autocatchment4326_182.0.shp,3.716205,6.434680,42.512530,52.663415,182.0,0.297805,0.000062
92,autocatchment4326_181.0.shp,3.729385,6.447753,42.514857,52.691994,181.0,0.298042,0.000237
...,...,...,...,...,...,...,...,...
120,autocatchment4326_4.0.shp,16.612905,33.741177,128.025866,178.379948,4.0,1.087785,0.000039
109,autocatchment4326_3.0.shp,16.667819,33.772396,128.033847,178.474062,3.0,1.088513,0.000728
98,autocatchment4326_2.0.shp,16.687347,33.887970,128.153225,178.728542,2.0,1.090469,0.001956
1,autocatchment4326_1.0.shp,16.694246,33.896155,128.153225,178.743625,1.0,1.090602,0.000133
